### Objective 

In this SQL practice project, we will be exploring the dataset,with the goal of answering questions and gaining insights from the data.Throughout the project, we will be focusing on improving our SQL skills, including writing efficient queries.

In [1]:

# # Importing necessary libraries

import numpy as np 
import pandas as pd 


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/soccer/database.sqlite


In [2]:
# Import the SQLite3 library and set the path variable to the directory.

import sqlite3
path = "../input/soccer/" 
database = path + 'database.sqlite'

In [3]:
# Establish a connection to the SQLite database

conn = sqlite3.connect(database)

# Retrieve information on all tables in the database using the 'sqlite_master' table
tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


### Country-wise list of leagues

In [4]:
country_league = pd.read_sql ("""
                                SELECT c.id,
                                       l.name AS league_name,
                                       c.name AS country_name
                                FROM league as l
                                JOIN country as c ON c.id = l.id
                                GROUP BY c.name, l.name""",conn)
country_league

,id,league_name,country_name
0,1,Belgium Jupiler League,Belgium
1,1729,England Premier League,England
2,4769,France Ligue 1,France
3,7809,Germany 1. Bundesliga,Germany
4,10257,Italy Serie A,Italy
5,13274,Netherlands Eredivisie,Netherlands
6,15722,Poland Ekstraklasa,Poland
7,17642,Portugal Liga ZON Sagres,Portugal
8,19694,Scotland Premier League,Scotland
9,21518,Spain LIGA BBVA,Spain


### List of Teams, Leagues, and their Countries.

In [5]:
team_leagues = pd.read_sql(""" WITH temp AS
                    (
                    SELECT t.team_long_name AS team_name,
                            l.name AS league_name,
                            l.id as league_id
                        
                    FROM league AS l 
                    JOIN match AS m on l.id = m.league_id
                    JOIN team AS t on m.home_team_api_id = t.team_api_id
                    GROUP BY l.name, t.team_long_name
                    )
                    SELECT t.team_name,t.league_name,c.name AS country
                    FROM temp AS t 
                    JOIN country AS c ON c.id = t.league_id
                    
                    
                    """,conn)
team_leagues.head(5)

,team_name,league_name,country
0,Beerschot AC,Belgium Jupiler League,Belgium
1,Club Brugge KV,Belgium Jupiler League,Belgium
2,FCV Dender EH,Belgium Jupiler League,Belgium
3,KAA Gent,Belgium Jupiler League,Belgium
4,KAS Eupen,Belgium Jupiler League,Belgium


### Match Results with Winning Team and Team Details.

In [6]:
 match_results = pd.read_sql("""  SELECT m.match_api_id, 
                                ht.team_long_name AS  home_team,
                                AT.team_long_name AS away_team,
                                home_team_goal, 
                                away_team_goal,
                                season,
                                stage,
                            CASE 
                                when m.home_team_goal > m.away_team_goal then ht.team_long_name
                                when m.home_team_goal < m.away_team_goal then at.team_long_name else 'Tie' end as winner
                            FROM Match AS m
                            LEFT JOIN Team AS ht on ht.team_api_id = m.home_team_api_id
                            LEFT JOIN Team AS at on at.team_api_id = m.away_team_api_id
                            """,conn)


In [7]:
match_results

,match_api_id,home_team,away_team,home_team_goal,away_team_goal,season,stage,winner
0,492473,KRC Genk,Beerschot AC,1,1,2008/2009,1,Tie
1,492474,SV Zulte-Waregem,Sporting Lokeren,0,0,2008/2009,1,Tie
2,492475,KSV Cercle Brugge,RSC Anderlecht,0,3,2008/2009,1,RSC Anderlecht
3,492476,KAA Gent,RAEC Mons,5,0,2008/2009,1,KAA Gent
4,492477,FCV Dender EH,Standard de Liège,1,3,2008/2009,1,Standard de Liège
...,...,...,...,...,...,...,...,...
25974,1992091,FC St. Gallen,FC Thun,1,0,2015/2016,9,FC St. Gallen
25975,1992092,FC Vaduz,FC Luzern,1,2,2015/2016,9,FC Luzern
25976,1992093,Grasshopper Club Zürich,FC Sion,2,0,2015/2016,9,Grasshopper Club Zürich
25977,1992094,Lugano,FC Zürich,0,0,2015/2016,9,Tie


### Most Successful Teams compared for all seasons


In [8]:
most_successful_team = pd.read_sql(""" WITH 
                        temp AS (

                            SELECT m.match_api_id, 
                                ht.team_long_name AS  home_team,
                                AT.team_long_name AS away_team,
                                home_team_goal, 
                                away_team_goal,
                                ht.team_api_id AS h_id,
                                at.team_api_id AS a_id,
                            CASE 
                                when m.home_team_goal > m.away_team_goal then ht.team_api_id
                                when m.home_team_goal < m.away_team_goal then at.team_api_id else 'Tie' end as winner
                            FROM Match AS m
                            LEFT JOIN Team AS ht on ht.team_api_id = m.home_team_api_id
                            LEFT JOIN Team AS at on at.team_api_id = m.away_team_api_id
                        ),
                        home AS (
                        
                            SELECT COUNT(match_api_id) AS home_games,
                                home_team AS team_name,
                                h_id
                            FROM temp
                            GROUP BY home_team 
                        ),
                        away AS (
                            SELECT COUNT(match_api_id) AS away_games,
                                away_team AS team_name,
                                a_id
                            FROM temp
                            GROUP BY away_team
                        ),
                        total AS (
                            SELECT a.away_games + h.home_games AS total_games,
                            h.team_name,
                            h.h_id
                            FROM home AS h 
                            JOIN away AS a ON h.h_id = a.a_id
                            GROUP BY h.team_name
                        ),
                        wins AS (
                            SELECT winner, COUNT(*) AS wins
                            FROM temp
                            GROUP BY winner 
                        )
                        
                        SELECT t.total_games,
                            t.team_name,
                            w.wins,
                            (w.wins)*100/(t.total_games) AS win_percentage
                        FROM total AS t
                        JOIN wins AS w on t.h_id = w.winner
                        ORDER BY wins DESC
                        
                    
                    
                    """,conn)

In [9]:
most_successful_team.head(10)

,total_games,team_name,wins,win_percentage
0,304,FC Barcelona,234,76
1,304,Real Madrid CF,228,75
2,304,Celtic,218,71
3,272,FC Bayern Munich,193,70
4,304,Manchester United,192,63
5,301,Juventus,189,62
6,248,SL Benfica,185,74
7,248,FC Porto,183,73
8,272,Ajax,181,66
9,286,FC Basel,180,62
